# <center>Simple LLM Based Workflow</center>

### Import modules

In [24]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

### Initialize Gemini model

In [ ]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")
    
model_name = "gemini-2.5-flash-lite"

# Make sure you have GOOGLE_API_KEY in your .env file
model = ChatGoogleGenerativeAI(
    model=model_name,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

Google API Key exists and begins AI


In [27]:
class LLMState(TypedDict):
    question: str
    answer: str


In [29]:
# response = model.chat.completions.create(model=model_name, messages=messages)

In [30]:
def llm_qa(state: LLMState) -> LLMState:
    question = state['question']
    
    # form a prompt
    prompt = f'Answer the following question {question}'
    
    # Ask that question to the LLM
    answer = model.invoke(prompt).content
    
    # update the answer in the state
    state['answer'] = answer
    
    return state

### Create graph and Add node

In [31]:
graph = StateGraph(LLMState)

### Add nodes

In [32]:
graph.add_node('llm_qa', llm_qa)

### Add edges

In [33]:
# First Edge
graph.add_edge(START, 'llm_qa')

#Second Edge
graph.add_edge('llm_qa', END)

### Compile

In [34]:
workflow = graph.compile()

### Execute

In [35]:
initial_state = {'question': "How far is moon from the earth?"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'How far is moon from the earth?', 'answer': "The distance between the Earth and the Moon isn't constant because the Moon's orbit around the Earth is elliptical, not a perfect circle.\n\nHowever, we can give you the **average distance**:\n\nThe average distance from the Earth to the Moon is approximately **384,400 kilometers (km)** or **238,900 miles**.\n\nHere's a little more detail:\n\n*   **Perigee:** The point in the Moon's orbit when it's closest to Earth. At perigee, the distance is about 363,300 km (225,700 miles).\n*   **Apogee:** The point in the Moon's orbit when it's farthest from Earth. At apogee, the distance is about 405,500 km (251,900 miles)."}
